In [1]:
import repo.utils.mnist_reader as mnist_reader

X_train, y_train = mnist_reader.load_mnist('repo/data/fashion', kind='train')
X_test, y_test = mnist_reader.load_mnist('repo/data/fashion', kind='t10k')

print(X_train.shape, y_train.shape)
print(X_test.shape, y_test.shape)


(60000, 784) (60000,)
(10000, 784) (10000,)


In [2]:
import numpy as np

num_data_points = len(X_train)

X_train_new = []
y_train_new = []

for i in range(num_data_points):
    if y_train[i] == 5:
        X_train_new.append(X_train[i])
        y_train_new.append(0)
    elif y_train[i] == 7:
        X_train_new.append(X_train[i])
        y_train_new.append(1)



X_train_new = np.array(X_train_new)
y_train_new = np.array(y_train_new)

X_train = X_train_new
y_train = y_train_new

X_train_og = X_train
y_train_og = y_train

print(X_train.shape)
print(y_train.shape)

(12000, 784)
(12000,)


In [3]:
num_samples_per_case = 3000

np.random.seed(42)

class_0_indices = np.where(y_train== 0)[0]
class_1_indices = np.where(y_train == 1)[0]

class_0_indices = np.random.choice(class_0_indices, num_samples_per_case, replace=False)
class_1_indices = np.random.choice(class_1_indices, num_samples_per_case, replace=False)

X_train_short = np.concatenate([X_train[class_0_indices], X_train[class_1_indices]])
y_train_short = np.concatenate([np.zeros(num_samples_per_case), np.ones(num_samples_per_case)])

indices = np.random.permutation(len(X_train_short))
X_train_short = X_train_short[indices]
y_train_short = y_train_short[indices]

X_train = X_train_short
y_train = y_train_short

print(X_train_short.shape)
print(y_train_short.shape)

# for i in range(6000):
    # print("Printing")
    # print(y_train_short[i])

# for i in range (6000):
#     if i == 3000:
#         print('halfway')
#     print(y_train[i])


(6000, 784)
(6000,)


In [4]:
import numpy as np

num_data_points = len(X_test)

X_test_new = []
y_test_new = []

for i in range(num_data_points):
    if y_test[i] == 5:
        X_test_new.append(X_test[i])
        y_test_new.append(0)
    elif y_test[i] == 7:
        X_test_new.append(X_test[i])
        y_test_new.append(1)

X_test_new = np.array(X_test_new)
y_test_new = np.array(y_test_new)

X_test = X_test_new
y_test = y_test_new

X_test_og = X_test
y_test_og = y_test

In [5]:
from sklearn.preprocessing import StandardScaler

# Assume X_train and X_test are your training and test feature matrices
scaler = StandardScaler()

# Fit on training data and transform training data
X_train = scaler.fit_transform(X_train)

# Transform test data using the same scaler (without refitting)
X_test = scaler.transform(X_test)


In [6]:
"""Select a probability in the range from 0.1 to 0.3 (e.g., you might pick 0.1, or 0.2, or 0.25,
 etc.). Fix this value, and call it p.
 Now, independently for each example in the training set, flip its label with probability p.
 This is called adding “label noise”. It makes the labels noisy and makes machine learning
 algorithms more likely to overfit"""

p = 0.2 # Probabilty

def flip_labels(y):
    y_noisy = y.copy()
    for i in range(len(y)):
        random_number = np.random.rand() #Random num between 0 and 1
        if random_number < p:
            y_noisy[i] = 1 - y_noisy[i]
    return y_noisy


y_train_noisy = flip_labels(y_train)


#Sanity check
same = 0
diff = 0
for i in range(len(y_train)):
    if y_train[i] == y_train_noisy[i]:
        same += 1
    else:
        diff += 1

print(same, diff)
print(diff / len(y_train))

4799 1201
0.20016666666666666


# Linear SVM

In [7]:
from sklearn.model_selection import train_test_split
from sklearn.svm import SVC
from sklearn.metrics import accuracy_score

kernal = 'linear'

c = 0.0005

svm = SVC(kernel=kernal, C=c)

svm.fit(X_train, y_train_noisy)

y_pred = svm.predict(X_test)

accuracy = accuracy_score(y_test, y_pred)
print(f'Accuracy: {accuracy * 100:.2f}%')



Accuracy: 94.50%


# Non Linear

In [8]:
from sklearn.model_selection import train_test_split
from sklearn.svm import SVC
from sklearn.metrics import accuracy_score

gamma = 0.001
c = 1.28

kernal = 'rbf'

svm = SVC(kernel=kernal, C=c, gamma=gamma)

svm.fit(X_train, y_train_noisy)

y_pred = svm.predict(X_test)

accuracy = accuracy_score(y_test, y_pred)
print(f'Accuracy: {accuracy * 100:.2f}%')



Accuracy: 96.45%


# Neural Network

In [9]:
from sklearn.neural_network import MLPClassifier

#Hyperparameters
hidden_layer_sizes = 30
activation = 'logistic'
learning_rate_init = 0.001

mlp = MLPClassifier(hidden_layer_sizes=hidden_layer_sizes, 
                    activation=activation,
                    learning_rate_init=learning_rate_init)                  

mlp.fit(X_train, y_train_noisy)

y_pred = mlp.predict(X_test)

accuracy = accuracy_score(y_test, y_pred)
print(f'Accuracy: {accuracy * 100:.2f}%')



Accuracy: 85.80%


c:\Users\qwebs\anaconda3\Lib\site-packages\sklearn\neural_network\_multilayer_perceptron.py:691: ConvergenceWarning: Stochastic Optimizer: Maximum iterations (200) reached and the optimization hasn't converged yet.
  warnings.warn(
